<a href="https://colab.research.google.com/github/JulienHelfenstein/World_model/blob/main/01_collect_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Définir le chemin racine de votre projet
PROJECT_ROOT = "/content/drive/My Drive/Colab Notebooks/World_model"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install gymnasium[box2d] numpy opencv-python tqdm

In [3]:
!pip install swig
!pip install "gymnasium[box2d]"

In [4]:
import gymnasium
import numpy as np
import cv2  # Bibliothèque OpenCV pour le redimensionnement
import os
from tqdm import tqdm # Pour une belle barre de progression

In [5]:
# --- 1. Configuration ---
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
N_EPISODES = 100       # Nombre d'épisodes à collecter
MAX_STEPS_PER_EPISODE = 300  # Limite de temps pour chaque épisode
IMG_SIZE = 64          # Taille cible pour notre CVAE (64x64)

# Créer le dossier de données s'il n'existe pas
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

# Fichier où les données seront sauvegardées
data_file = os.path.join(DATA_DIR, "carracing_data.npz")

print(f"Collection de {N_EPISODES} épisodes pour CarRacing...")
print(f"Les données seront sauvegardées dans : {data_file}")

Collection de 100 épisodes pour CarRacing...
Les données seront sauvegardées dans : /content/drive/My Drive/Colab Notebooks/World_model/data/carracing_data.npz


In [6]:
# --- 2. Fonction de pré-traitement de l'image ---
def preprocess_image(image):
    """
    Redimensionne l'image de 96x96x3 à 64x64x3
    et normalise les pixels entre 0.0 et 1.0.
    """
    # Redimensionner l'image
    resized_image = cv2.resize(image, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
    # Normaliser les pixels
    normalized_image = resized_image.astype(np.float32) / 255.0
    return normalized_image

In [7]:
# --- 3. Initialisation de l'environnement ---
# 'continuous=True' est la valeur par défaut, juste pour être explicite
# 'render_mode="human"' si vous voulez voir la voiture jouer (plus lent)
env = gymnasium.make('CarRacing-v3', continuous=True)

# Listes pour stocker toutes nos données
all_observations = []
all_actions = []

In [8]:
# --- 4. Boucle de collecte de données ---
try:
    # tqdm crée une barre de progression pour nos épisodes
    for episode in tqdm(range(N_EPISODES)):
        obs, info = env.reset()

        for step in range(MAX_STEPS_PER_EPISODE):
            # --- "Smarter" Random Policy ---
            # Au lieu d'une action 100% aléatoire, on force l'accélération
            # pour que la voiture explore davantage.
            steer = np.random.uniform(-1.0, 1.0)  # Volant aléatoire
            gas = 0.7  # Toujours accélérer !
            brake = 0.0  # Ne jamais freiner

            action = np.array([steer, gas, brake], dtype=np.float32)

            # 1. Prendre l'action dans l'environnement
            next_obs, reward, terminated, truncated, info = env.step(action)

            # 2. Pré-traiter l'observation (image)
            processed_obs = preprocess_image(next_obs)

            # 3. Stocker l'observation et l'action
            all_observations.append(processed_obs)
            all_actions.append(action)

            obs = next_obs

            # Si l'épisode est terminé (crash ou fin)
            if terminated or truncated:
                break

finally:
    env.close() # Toujours fermer l'environnement
    print("\nEnvironnement fermé.")

100%|██████████| 100/100 [07:07<00:00,  4.27s/it]


Environnement fermé.


In [9]:
# --- 5. Sauvegarde des données ---
if len(all_observations) > 0:
    print(f"Collecte terminée. Total d'images : {len(all_observations)}")

    # Convertir les listes en arrays NumPy
    obs_array = np.array(all_observations, dtype=np.float32)
    act_array = np.array(all_actions, dtype=np.float32)

    # Sauvegarder dans un fichier .npz compressé
    # C'est BEAUCOUP plus efficace que 30 000 fichiers PNG
    np.savez_compressed(data_file, observations=obs_array, actions=act_array)

    print(f"Données sauvegardées avec succès dans {data_file}")
    print(f"Shape des observations : {obs_array.shape}")
    print(f"Shape des actions : {act_array.shape}")
else:
    print("Aucune donnée n'a été collectée.")

Collecte terminée. Total d'images : 29878
Données sauvegardées avec succès dans /content/drive/My Drive/Colab Notebooks/World_model/data/carracing_data.npz
Shape des observations : (29878, 64, 64, 3)
Shape des actions : (29878, 3)
